In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import mysql.connector
import gc
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from pandas.core import datetools
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

/home/gspc/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/gspc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_connection():
    cnx2 = mysql.connector.connect(host='localhost',
                                   user='root', password='MyNewPass',
                                   database='Weather_Data')
    return cnx2

In [3]:
def select_data():
    conn = create_connection()
    cursor = conn.cursor()
    print('Extracting Data')
    query = "Select * from weather where Observation_date <= '2018-06-01 00:00:00';"
    cursor.execute(query, )
    result = cursor.fetchall()
    print('Extracted Data')
        
    cursor.close()
    conn.close()
    
    return result
    

In [4]:
def null_values(data):
    print(data.isnull().sum())

In [5]:
def change_structure(data):
    cols = ['date', 'rain', 'temp', 'wetb', 'dewpt','vappr', 'rhum', 'msl', 'wdsp', 'wddir','height','latitude', 'longitude', 'station','county']
    data = pd.DataFrame(data, columns=cols)
    
    #null_values(data)
    data.fillna(0, inplace=True)
    return data

In [6]:
def split_time(data):
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year.astype(np.uint16)
    data['month'] = data['date'].dt.month.astype(np.uint8)
    data['day'] = data['date'].dt.day.astype(np.uint8)
    data['hour'] = data['date'].dt.hour.astype(np.uint8)
    
    return data

In [7]:
def generate_season(data):
    data['season'] = pd.cut(data['month'], bins=[0,1,4,7,10,14], labels=['Winter','Spring','Summer','Autumn','Winter2']).str.replace('Winter2','Winter')
    data['season'] = data['season'].astype('category')
    return data

In [8]:
def time_features_to_categorical(data):
    #data['year'] = data['year'].astype(str)
    #data['month'] = data['month'].astype(str)
    #data['day'] = data['day'].astype(str)
    #data['hour'] = data['hour'].astype(str)
    
    data['year'] = data['year'].astype('category')
    data['month'] = data['month'].astype('category')
    data['day'] = data['day'].astype('category')
    data['hour'] = data['hour'].astype('category')
    
    return data

In [9]:
def convert_wdsp(data):
    data['wdsp'] = data['wdsp'] * 1.852
    
    return data

In [10]:
def update_rhum_values(data):
    print('Updating relative humidity values <= 0')
    rhum_mean = data['rhum'].mean()
    data.loc[data['rhum'] <= 0, 'rhum'] = rhum_mean
    
    return data

In [11]:
def update_wetb_values(data):
    print('Updating wet bulb air temperature values <= 0')
    wetb_mean = data['wetb'].mean()
    data.loc[data['wetb'] <= -40, 'wetb'] = wetb_mean
    
    return data

In [12]:
def update_dewpt_values(data):
    print('Updating dew point air temperature values <= 0')
    dewpt_mean = data['dewpt'].mean()
    data.loc[data['dewpt'] <= -20,'dewpt'] = dewpt_mean
    
    return data

In [13]:
def update_msl_values(data):
    print('Updating mean sea level pressure values <= 0')
    msl_mean = data['msl'].mean() 
    data.loc[data['msl'] < 940, 'msl'] = msl_mean
    
    return data

In [14]:
def update_vappr_values(data):
    print('Updating vapour pressure values <= 0')
    vappr_mean = data['vappr'].mean()
    data.loc[data['vappr'] <= 0, 'vappr'] = vappr_mean
    
    return data

In [15]:
def binarize_categories(data):
    print('Binarizing categorical data')
    
    categorical_columns = []

    for col in data.columns:
        if isinstance(data[col][0], str):
            print('Changing ', col, ' to categorical')
            categorical_columns.append(col)
            encoder = LabelBinarizer()
            data[col] = encoder.fit_transform(data[col])
            
    #data = delete_original_categories(categorical_columns, data)        
    return data

In [16]:
def delete_original_categories(categorical_columns, data):
    print('Deleting original categories')
    for col in categorical_columns:
        if col in data.columns:
            del(data[col])
            gc.collect()
    return data

In [17]:
def remove_unwanted_features(data):
    print('Removing features')
    data = data.drop( columns = ['date', 'height', 'latitude', 'longitude', 'station', 'county'])
    
    return data

In [18]:
def time_lagged_features(data, shift_freq, shift_name):
    print('Generating time lagged features')
    
    temp = data
    temp.drop(columns=['season'], inplace=True)
    
    cols = ['year', 'month', 'day', 'hour', 'rain', 'temp', 'wetb', 'dewpt','vappr', 'rhum', 'msl', 'wdsp', 'wddir']
    new_cols = []
    
    for item in cols:
        item = item + shift_name 
        new_cols.append(item)
    
    temp = temp.shift(shift_freq)
    temp.columns = new_cols
    
    temp.reset_index(drop=True, inplace=True)
    data.reset_index(drop=True, inplace=True)
    data = data.join(temp)

    return data

In [19]:
def sort_data(data):
    data = data.sort_values(['year','month','day','hour'])
    return data

In [20]:
def aggregate_data(data):
    #may need to drop season feature then recreate it
    #['rain','temp', 'wetb', 'dewpt', 'vappr', 'rhum', 'msl', 'wdsp', 'wddir']
    print('Aggregating Data')
    season = data['season']
    data = data.groupby( by=['year','month','day','hour']).mean().reset_index()
    sorted_data = sort_data(data)
    data['season'] = season
    
    print(len(sorted_data),' observations available for training')
    
    return sorted_data

In [21]:
def convert_to_supervised(data):
    temp = data
    temp = temp['rain'].shift(-1)
    
    data['target_rainfall'] = temp
    
    data.drop(data.index[len(data)-1], inplace=True)
    data.drop(data.index[0], inplace=True)
    data.fillna(data.mean(), inplace=True)    
    data.reset_index(inplace=True, drop=True)
    
    return data

In [22]:
def normalise_data(data):
    print('Normalising test data')
    data.drop(columns = ['target_rainfall'], inplace=True)
    column_names = data.columns
    scaler = MinMaxScaler()
    normalised_data = scaler.fit_transform(data)

    normalised_data = pd.DataFrame(normalised_data, columns=column_names)
    
    return [normalised_data, scaler]

In [23]:
def train_test_split(data):
    print('Getting test set')
    
    #validate on last 30 days without retraining model after each prediction
    test_data = -30
    
    
    train = data[:test_data]
    test = data[test_data:]
    
    Xtrain, Ytrain = train.loc[:, train.columns != 'rain'], train.loc[:, train.columns == 'rain']
    Xtest, Ytest = test.loc[:, test.columns != 'rain'], test.loc[:, test.columns == 'rain']
    
    Xtrain = Xtrain.values.astype('float32') # convert to numpy array
    Xtest = Xtest.values.astype('float32')
    
    Xtrain = Xtrain.reshape((Xtrain.shape[0], 1, Xtrain.shape[1]))
    #Xtest = Xtest.reshape((Xtest.shape[0], 1, Xtest.shape[1]))
    
    print(Xtrain.shape)
    print(Xtest.shape)
    
    return [Xtrain, Ytrain, Xtest, Ytest]

In [24]:
def train_lstm(Xtrain, Ytrain, Xtest, Ytest):
    # initialise train model
    model = Sequential()
    model.add(LSTM(100, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    model.add(Dense(1)) #determines the number of outputs
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # batch size determines how many input observations the network sees before it can update weigths
    model.fit(Xtrain, Ytrain, epochs=50, batch_size=100, verbose=1, shuffle=False)
    
    train_weigths = model.get_weights() # get weights of trained model to use for initialisation of prediction model
    
    # initialise test model
    
    test_model = Sequential()
    test_model.add(LSTM(100, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    test_model.add(Dense(1)) #determines the number of outputs
    test_model.compile(loss='mean_squared_error', optimizer='adam')
    test_model.set_weights(train_weigths)
    
    return test_model

In [66]:
def test_lstm(test_model, Xtest, Ytest, scaler):
    Ytest = Ytest.values
    predicted_output = []
    Xtest = Xtest.reshape(len(Xtest),1,25)
    predicted_output.append(test_model.predict(Xtest, batch_size=len(Xtest)))
    print('RMSE: ', math.sqrt(mean_squared_error(Ytest, predicted_output[0])))
    
    #Code for sequential predictions
    '''for i in range(len(Xtest)):
        x_input = Xtest[i]
        y_output = Ytest[i]
        x_input = x_input.reshape(1,1,25)
        predicted_output.append(test_model.predict(x_input, batch_size=len(Xtest)))
        print(predicted_output)
        
        #print('RMSE: ', math.sqrt(mean_squared_error(y_output, predicted_output)))
        #print('Original value: ', y_output, ' Predicted value: ', predicted_output)'''

In [26]:
data = select_data()
data = change_structure(data)
data = split_time(data)
data = generate_season(data)
#data = time_features_to_categorical(data)
data = remove_unwanted_features(data)
data = convert_wdsp(data)
data = sort_data(data)
data = binarize_categories(data)

data = update_rhum_values(data)
data = update_wetb_values(data)
data = update_dewpt_values(data)
data = update_msl_values(data)
data = update_vappr_values(data)

data.reset_index(drop=True, inplace=True)
data = aggregate_data(data)
data = time_lagged_features(data, 1, '1hr')

data.head()

Extracting Data
Extracted Data
Removing features
Binarizing categorical data
Changing  season  to categorical
Updating relative humidity values <= 0
Updating wet bulb air temperature values <= 0
Updating dew point air temperature values <= 0
Updating mean sea level pressure values <= 0
Updating vapour pressure values <= 0
Aggregating Data
266617  observations available for training
Generating time lagged features


,year,month,day,hour,rain,temp,wetb,dewpt,vappr,rhum,msl,wdsp,wddir,year1hr,month1hr,day1hr,hour1hr,rain1hr,temp1hr,wetb1hr,dewpt1hr,vappr1hr,rhum1hr,msl1hr,wdsp1hr,wddir1hr
0,1988,1,1,0,0.14,7.90,6.93,5.74,9.22,86.30,997.62,29.82,229.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1988,1,1,1,0.30,7.93,7.14,6.17,9.50,88.80,997.18,31.11,230.00,"1,988.00",1.00,1.00,0.00,0.14,7.90,6.93,5.74,9.22,86.30,997.62,29.82,229.00
2,1988,1,1,2,0.73,8.03,7.50,6.86,9.97,92.40,996.52,30.19,226.00,"1,988.00",1.00,1.00,1.00,0.30,7.93,7.14,6.17,9.50,88.80,997.18,31.11,230.00
3,1988,1,1,3,0.36,8.35,7.88,7.34,10.29,93.30,995.71,30.93,220.00,"1,988.00",1.00,1.00,2.00,0.73,8.03,7.50,6.86,9.97,92.40,996.52,30.19,226.00
4,1988,1,1,4,0.19,8.74,8.25,7.68,10.56,93.40,994.84,30.74,218.00,"1,988.00",1.00,1.00,3.00,0.36,8.35,7.88,7.34,10.29,93.30,995.71,30.93,220.00


In [27]:
data = convert_to_supervised(data)

In [28]:
normalised_data = normalise_data(data)

Normalising test data


In [29]:
%time lstm_data = train_test_split(normalised_data[0])

Getting test set
(266585, 1, 25)
(30, 25)
CPU times: user 20 ms, sys: 16 ms, total: 36 ms
Wall time: 31.9 ms


In [30]:
%time test_model = train_lstm(lstm_data[0], lstm_data[1], lstm_data[2], lstm_data[3])

Epoch 1/50
266585/266585 [==============================] - 9s 33us/step - loss: 0.0012
Epoch 2/50
266585/266585 [==============================] - 8s 30us/step - loss: 9.5327e-04
Epoch 3/50
266585/266585 [==============================] - 8s 30us/step - loss: 9.3034e-04
Epoch 4/50
266585/266585 [==============================] - 8s 30us/step - loss: 9.1219e-04
Epoch 5/50
266585/266585 [==============================] - 8s 30us/step - loss: 9.0116e-04
Epoch 6/50
266585/266585 [==============================] - 8s 30us/step - loss: 8.9425e-04
Epoch 7/50
266585/266585 [==============================] - 8s 30us/step - loss: 8.8939e-04
Epoch 8/50
266585/266585 [==============================] - 8s 30us/step - loss: 8.8536e-04
Epoch 9/50
266585/266585 [==============================] - 8s 30us/step - loss: 8.8197e-04
Epoch 10/50
266585/266585 [==============================] - 8s 30us/step - loss: 8.7920e-04
Epoch 11/50
266585/266585 [==============================] - 8s 30us/step - loss: 8

In [67]:
test_lstm(test_model, lstm_data[2], lstm_data[3], normalised_data[1])

RMSE:  0.010577342755437966


# Important steps

Shift data so its supervised learning problem 

Aggregate data (for future iterations)

Normalise data using standard scaler

### Xgboost 1st Run (Try on cpu and gpu)

Raw features only - remove height, station, county, longitude and latitude

### Xgboost 2nd Run

Shift features to include previous hour. Will first only include the rainfall from the previous hour

### Xgboost 3rd Run

Shift features to include 2 hours previous 

### Xgboost 3rd Run

Shift features to include 1 **and** 2 hours previous 


## Potential Features
minus the monthly, yearly , seasonal average, lagged features, median values?

aggregate data, difference between current and previous values

num splits equal to num distinct years by 12 (months)

31 * 12